In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession.\
builder.\
config('spark.ui.port','0').\
config("spark.sql.warehouse.dir",f"/user/itv012676/warehouse").\
enableHiveSupport().\
master('yarn').\
getOrCreate()

we need to consider 3 datasets

orders
========
order_id,order_date,order_customer_id,order_status

customers
==========
customer_id,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode

order_items
==============
order_item_id,order_id,order_item_product_id,order_item_quantity,order_item_subtotal,order_item_product_price


1.we  need to find top10 customers who have spent the most  amount (premium customers) 

2.top10 productid's with most quantities sold 

3.how many customers are from Caguascity

4.Top3 states with maximum customers

5.how many customers have spent more than $1000 in total


6.which state has most number of orders in CLOSED status 

7.how many customers are active(active customers are the one's who placed atleast oneorder) 

8.What is the revenue generated by each state in sorted order

In [2]:
spark

In [3]:
order_items_rdd = spark.sparkContext.textFile("/public/trendytech/retail_db/order_items/*")
orders_rdd = spark.sparkContext.textFile("/public/trendytech/retail_db/orders/*")
customers_rdd = spark.sparkContext.textFile("/public/trendytech/retail_db/customers/*")

In [4]:
order_items_rdd.take(5)

['1,1,957,1,299.98,299.98',
 '2,2,1073,1,199.99,199.99',
 '3,2,502,5,250.0,50.0',
 '4,2,403,1,129.99,129.99',
 '5,4,897,2,49.98,24.99']

In [5]:
orders_rdd.take(5)

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE']

In [6]:
customers_rdd.take(5)

['1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521',
 '2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126',
 '3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer Bend,Caguas,PR,00725',
 '4,Mary,Jones,XXXXXXXXX,XXXXXXXXX,8324 Little Common,San Marcos,CA,92069',
 '5,Robert,Hudson,XXXXXXXXX,XXXXXXXXX,"10 Crystal River Mall ",Caguas,PR,00725']

In [7]:
mapped_order_items_rdd_1 = order_items_rdd.map(lambda x:(x.split(",")[1],x.split(",")[4]))

In [8]:
customers_mapped_1 = orders_rdd.map(lambda x:(x.split(",")[0],x.split(",")[2]))

In [9]:
joined_rdd_1 = mapped_order_items_rdd_1.join(customers_mapped_1)
joined_rdd_1.take(15)

[('35212', ('49.98', '8774')),
 ('35212', ('299.97', '8774')),
 ('35212', ('249.9', '8774')),
 ('35212', ('49.98', '8774')),
 ('35212', ('149.94', '8774')),
 ('35214', ('129.99', '388')),
 ('35214', ('149.94', '388')),
 ('35214', ('399.98', '388')),
 ('35214', ('239.96', '388')),
 ('35214', ('399.98', '388')),
 ('35215', ('199.99', '11572')),
 ('35215', ('199.92', '11572')),
 ('35222', ('129.99', '9654')),
 ('35223', ('99.96', '1505')),
 ('35223', ('74.97', '1505'))]

In [10]:
customer_sub_total_mapped = joined_rdd_1.map(lambda x: (int(x[1][1]),float(x[1][0])))

In [11]:
premium_customers = customer_sub_total_mapped.reduceByKey(lambda x,y:x+y)

In [12]:
sorted_customers = premium_customers.sortBy(lambda x:x[1],False).take(10)

In [13]:
sorted_customers #final answer for first question

[(791, 10524.17),
 (9371, 9299.03),
 (8766, 9296.14),
 (1657, 9223.71),
 (2641, 9130.92),
 (1288, 9019.11),
 (3710, 9019.099999999999),
 (4249, 8918.85),
 (5654, 8904.95),
 (5624, 8761.98)]

In [14]:
mapped_order_items_rdd_2 = order_items_rdd.map(lambda x:(int(x.split(",")[2]),int(x.split(",")[3])))
mapped_order_items_rdd_2.take(10)

[(957, 1),
 (1073, 1),
 (502, 5),
 (403, 1),
 (897, 2),
 (365, 5),
 (502, 3),
 (1014, 4),
 (957, 1),
 (365, 5)]

In [15]:
order_items_reduced_rdd_2 = mapped_order_items_rdd_2.reduceByKey(lambda x,y:x+y)
sorted_items_rdd = order_items_reduced_rdd_2.sortBy(lambda x:x[1],False)
sorted_items_rdd.take(10) #final answer for second question

[(365, 73698),
 (502, 62956),
 (1014, 57803),
 (191, 36680),
 (627, 31735),
 (403, 22246),
 (1004, 17325),
 (1073, 15500),
 (957, 13729),
 (977, 998)]

In [16]:
customers_filtered_rdd_3 = customers_rdd.filter(lambda x:x.split(",")[6]=="Caguas")
customers_filtered_rdd_3.count()#final answer for third question

4584

In [17]:
customers_mapped_rdd_4 = customers_rdd.map(lambda x:(x.split(",")[7],1))

In [18]:
customers_state_wise = customers_mapped_rdd_4.reduceByKey(lambda x,y:x+y)


In [19]:
top_states = customers_state_wise.sortBy(lambda x:x[1],False).take(3) 
top_states #final answer for 4th question

[('PR', 4771), ('CA', 2012), ('NY', 775)]

In [20]:
customers_spent_greaterthan_1000 = premium_customers.filter(lambda x: x[1]>1000)
customers_spent_greaterthan_1000.count() #final answer for 5th question

11148

In [22]:
orders_filtered_rdd = orders_rdd.filter(lambda x:x.split(",")[3]=="CLOSED")
orders_mapped_rdd_6 = orders_filtered_rdd.map(lambda x:(x.split(",")[2],x.split(",")[3]))
customers_state_mapped = customers_rdd.map(lambda x:(x.split(",")[0],x.split(",")[7]))
closed_states_rdd = customers_state_mapped.join(orders_mapped_rdd_6)
closed_states_mapped_rdd = closed_states_rdd.map(lambda x:(x[1][0],1))
state_with_high_closed_orders = closed_states_mapped_rdd.reduceByKey(lambda x,y :x+y).sortBy(lambda x:x[1],False).take(1)
state_with_high_closed_orders #final answer for 6th question

[('PR', 2891)]

In [23]:
orders_mapped_rdd_7 = orders_rdd.map(lambda x:(x.split(",")[2],1))
active_customers_rdd = orders_mapped_rdd_7.reduceByKey(lambda x,y :x+y)
active_customers_rdd.count() #final answer for 7th question

12405

In [25]:
orders_mapped_rdd_8 = orders_rdd.map(lambda x:(int(x.split(",")[2]),int(x.split(",")[0])))
customers_state_mapped = customers_rdd.map(lambda x:(int(x.split(",")[0]),x.split(",")[7]))
customers_orders_joined_rdd = orders_mapped_rdd_8.join(customers_state_mapped)

orderid_states_mapped = customers_orders_joined_rdd.map(lambda x:(x[1][0],x[1][1]))
order_items_mapped_rdd_8 = order_items_rdd.map(lambda x:(int(x.split(",")[1]),float(x.split(",")[4])))
orderid_states_revenue_joined_rdd = orderid_states_mapped.join(order_items_mapped_rdd_8)
state_revenue_mapped = orderid_states_revenue_joined_rdd.map(lambda x:(x[1][0],x[1][1]))
top10_states_revenue = state_revenue_mapped.reduceByKey(lambda x,y :x+y).sortBy(lambda x:x[1],False).take(10)
top10_states_revenue

[('PR', 13208867.68999927),
 ('CA', 5542722.999999756),
 ('NY', 2152706.7399999835),
 ('TX', 1731407.489999999),
 ('IL', 1457225.8300000029),
 ('FL', 1048609.7700000026),
 ('OH', 773804.1100000018),
 ('MI', 730078.9700000018),
 ('PA', 724375.9300000016),
 ('NJ', 606550.990000001)]

Cases
=====
date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,recovered,dataQualityGrade,lastUpdateEt,dateModified,checkTimeEt,death,hospitalized,dateChecked,totalTestsViral,positiveTestsViral,negativeTestsViral,positiveCasesViral,deathConfirmed,deathProbable,fips,positiveIncrease,negativeIncrease,total,totalTestResults,totalTestResultsIncrease,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade

States
=====
state,notes,covid19Site,covid19SiteSecondary,covid19SiteTertiary,twitter,covid19SiteOld,name,fips,pui,pum

1.Find the top-10 states with the highest no.of positive cases.

2.Find the total count of people in ICU currently.

3.Find the top-15 States having maximum no.of recovery

4.Find the top3 States having least no.of deaths

5.Find the total number of people hospitalized currently.

6.List the twitter handle and fipscode for the top15 states with the highest number of total cases.


In [26]:
cases_rdd = spark.sparkContext.textFile("/public/trendytech/covid19/cases/*")
states_rdd = spark.sparkContext.textFile("/public/trendytech/covid19/states/*")


In [27]:
cases_mapped_rdd_1 = cases_rdd.map(lambda x: (x.split(",")[1],int(x.split(",")[2])))
top_10_states = cases_mapped_rdd_1.reduceByKey(lambda x,y: x+y).sortBy(lambda x:x[1],False).take(10)
top_10_states #final solution for question 1

[('WA', 1701),
 ('GA', 1017),
 ('MH', 730),
 ('MI', 61),
 ('CA', 53),
 ('GJ', 35),
 ('BR', 23),
 ('JH', 13),
 ('CG', 8),
 ('RI', 6)]

In [28]:
patientsinICU_rdd_2 = cases_rdd.map(lambda x:int(x.split(",")[7]))
patientsinICU_rdd_2.sum()#final solution for question 2

1344

In [29]:
cases_mapped_rdd_3 = cases_rdd.map(lambda x: (x.split(",")[1],int(x.split(",")[11])))
top_15_states_recovery = cases_mapped_rdd_3.reduceByKey(lambda x,y: x+y).sortBy(lambda x:x[1],False).take(15)
top_15_states_recovery #final solution for question 3

[('WA', 451),
 ('MH', 165),
 ('MI', 101),
 ('GA', 87),
 ('AP', 84),
 ('RI', 72),
 ('BR', 68),
 ('JH', 50),
 ('KA', 43),
 ('AZ', 38),
 ('AS', 30),
 ('GJ', 27),
 ('CA', 23),
 ('HR', 20),
 ('HP', 19)]

In [30]:
cases_mapped_rdd_4 = cases_rdd.map(lambda x: (x.split(",")[1],int(x.split(",")[23])))
top_3_least_deaths = cases_mapped_rdd_4.reduceByKey(lambda x,y: x+y).sortBy(lambda x:x[1]).take(3)
top_3_least_deaths #final solution for question 4

[('AS', 9), ('JH', 10), ('CG', 31)]

In [31]:
hospitalized_rdd_5 = cases_rdd.map(lambda x:int(x.split(",")[5]))
hospitalized_rdd_5.sum()#final solution for question 5

1319

In [33]:
states_mapped_rdd_6 = states_rdd.map(lambda x:(x.split(",")[0],(x.split(",")[5],x.split(",")[8])))
total_cases_mapped = cases_rdd.map(lambda x:(x.split(",")[1],int(x.split(",")[28])))
total_cases_reduced = total_cases_mapped.reduceByKey(lambda x,y:x+y)
twitter_states_joined_rdd = total_cases_reduced.join(states_mapped_rdd_6)
twitter_states_joined_sorted_rdd = twitter_states_joined_rdd.sortBy(lambda x:x[1][0],False)
twitter_states_joined_sorted_rdd.take(15) #final answer for question 6

[('WA', (2100, ('@WACovid', '44'))),
 ('GA', (1034, ('@GACovid', '44'))),
 ('MH', (730, ('@MHCovid', '26'))),
 ('CA', (515, ('@CACovid', '4'))),
 ('MI', (61, ('@MICovid', '53'))),
 ('GJ', (35, ('@GJCovid', '44'))),
 ('AZ', (34, ('@AZCovid', '53'))),
 ('BR', (23, ('@BRCovid', '53'))),
 ('RI', (16, ('@RICovid', '26'))),
 ('JH', (13, ('@JHCovid', '53'))),
 ('CG', (8, ('@CGCovid', '53'))),
 ('KA', (5, ('@KACovid', '53'))),
 ('HP', (4, ('@HPCovid', '53'))),
 ('AS', (2, ('@ASCovid', '6'))),
 ('HR', (2, ('@HRCovid', '9')))]

Given Trendytech Students GoogleReviews
You need to find the top-20 words This is done with a thought process that the final answer would represent the central theme of the reviews,like what most of the students are saying. But there would be boring words like "the", "is", "are", "a" etc.. which do not have any significance.
Your solution should not consider such boringwords, so you need to remove the boring words and give the top-20 keywords which come the most of the time.
The list of boring words is kept in a file called boringwords.txt

In [46]:
reviews_rdd = spark.sparkContext.textFile("/public/trendytech/reviews/trendytech-student-reviews.csv")
boring_words_rdd = spark.sparkContext.textFile("/user/itv012676/TextAnalysis/boringwords.txt")
reviews_preprocessed_rdd = reviews_rdd.flatMap(lambda x:x.split(" ")).map(lambda x:x.lower())
boring_words = boring_words_rdd.collect()
reviews_filtered_rdd = reviews_preprocessed_rdd.map(lambda x:(x,x not in boring_words)).filter(lambda x: x[1]==True).map(lambda x:(x[0],1)).reduceByKey(lambda x,y:x+y).sortBy(lambda x:x[1],False)
reviews_filtered_rdd.take(20)

[('data', 201),
 ('sumit', 109),
 ('trendytech', 67),
 ('', 64),
 ('data.', 34),
 ('course.', 33),
 ("sir's", 23),
 ('trendy', 14),
 ('course,', 13),
 ("master's", 13),
 ('domain.', 12),
 ("trendytech's", 12),
 ('sir.', 11),
 ('program.', 9),
 ('concepts.', 9),
 ('field.', 9),
 ('hands-on', 8),
 ('fresher', 8),
 ('amazing.', 8),
 ('career.', 7)]